In [ ]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year


## Field Data

In [ ]:
field_data = ee.FeatureCollection(f"{data_folder}/field")

# Reproject continuous variables using mean
continuous_vars = [
    nearest_mature.rename("nearest_mature"),
    soil,
    terraclim.select(["mean_soil", "mean_vpd", "mean_temp", "mean_def", 
                      "mean_srad", "mean_pr", "mean_aet", "mean_pdsi"])
]

unified_data_field = ee.Image.cat([
    floodable_forests,
    distance_deep_forest,
    sur_cover,
    continuous_vars,
    categorical_vars,
    ee.Image.pixelLonLat().rename(['lon', 'lat'])
])

unified_fc = unified_data_field.reduceRegions(field_data, ee.Reducer.first(), 30)

properties_to_export = unified_data_field.bandNames().getInfo()
properties_to_export = properties_to_export + ["age", "site_id", "plot_id", "field_biom"]

# Export task to Google Drive
task = ee.batch.Export.table.toDrive(
    collection = unified_fc,
    description = 'field_predictors',
    fileFormat = "CSV",
    selectors = [p for p in properties_to_export if p not in ['system:index', '.geo']]
)
task.start()